# Chapter 2: Explore STAC Catalog and Read NetCDF files using STAC
* **Creators:** Ann Windnagel and Robyn Marowitz
* **Affiliation:** [National Snow and Ice Data Center (NSIDC)](https://nsidc.org/home "National Snow and Ice Data Center")
* **History:**
  * January 2025 - Initial version

## Overview

In this notebook (Notebook 2), you will learn what a SpatioTemporal Asset Catalog (STAC) is. You will also learn how to explore the features and variables of the SIC CDR STAC Collection and Items and open and read the SIC CDR netCDF files using PySTAC.

### Prerequisites
Read and understand the material in Notebook 1.

### Targeted Level
Beginner

### Learning Outcomes
Once you have completed this notebook, you will be able to:
* Open the SIC CDR STAC collection and extract the content list using `PySTAC`
* Explore the *Collection* and its *Items*, properties, and metadata
* Open and read a SIC CDR netCDF file from the STAC using `Xarray`

### Time Estimate
TBD

## What is STAC?

A [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/en "Spatio Temporal Asset Catalog Web Site") improves the accessibility and discoverability of the data it describes. A STAC catalog provides special metadata about the collection that provides a standardized way to expose the spatio-temporal data and allows it to be more easily worked with, indexed, and discovered. 

A STAC catalog uses a JSON (JavaScript Object Notation) schema to describe the data via different STAC specifications. For the SIC CDR STAC, NSIDC used the *Collection* and *Item* specifications. The *Collection* specification is used to describe a group of related *Items* using a set of common metadata fields. In this case, the *Collection* is the entire SIC CDR data product. The *Item* specification is used to describe each asset of the *Collection*. In this case, each individual SIC CDR netCDF file. Thes two specifications provide product-level (*Collection*) and file-level (*Item*) metadata for the data set.

This SIC CDR STAC catalog begins with a JSON file that defines the *Collection* in a file called `collection.json`. Each *Item* represents a single spatio-temporal asset of the catalog, and each is described in its own JSON file. For the SIC CDR, there is one *Item* file for each daily and monthly aggregated netCDF file for the northern and southern hemispheres.

<div class="alert alert-block alert-success">
<h3><strong>Explore</strong></h3>
<h4>Check out the STAC JSON files</h4>
If you would like to view the sea ice CDR STAC JSON files before you work with them programmatically, you can see them here: <a href="https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/" title="Sea Ice Concentration Climate Data Record STAC Catalog">https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/</a>
</div>

### Import Relevant Packages
You will use [pystac](https://pystac.readthedocs.io/en/stable/ "Pystac documentation") to explore the sea ice CDR STAC *Collection* and *Items* and to view their details. [Xarray](https://docs.xarray.dev/en/stable/ "Xarray documentation") makes working with labelled multi-dimensional arrays, such as netCDF, in Python simple and efficient. [Fsspec](https://filesystem-spec.readthedocs.io/en/latest/ "Fsspec documentation") is a pythonic file system interface to make it easier to interact with file systems.

In [1]:
import xarray as xr
import fsspec
import pystac

************************
## Tutorial Part 1

Read and explore STAC Collection
*************************

### Read the STAC Collection File

To begin, you will read in the sea ice CDR STAC `collection.json` file and assign it to a variable `collection` so that you can explore it using `pystac`.

In [2]:
# Specify the URL to the sea ice CDR collection.json file on NSIDC servers
collection_url = 'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json'

# Read file and assign to a variable
collection = pystac.Collection.from_file(collection_url)

### Explore Collection Information

In the following cells, you will look at how the *Collection* is organized and some of its metadata attributes. 

#### List All *Items* in the *Collection*
Using the `describe` method on the `collection` variable created above, you can display all of the *Items* for this *Collection*. The sea ice CDR data spans 1978 through the present year. You will see an item listed for each year and hemisphere for the daily files, plus one monthly file for each hemisphere that contains the whole time series.

In the `Item id`, there is a wealth of information due to its consistent format:

`seaice_conc_TIME_HH_YYYY[MM]_VXXRXX` 

where TIME is `daily` or `monthly`, HH is the hemisphere and can be `nh` for Northern Hemisphere or `sh` for Southern Hemisphere, YYYY is the 4-digit year of the data, MM is the 2-digit month (used for the monthly files only), and VXXRXX is the version (you will use version v04r00 in these notebooks).

<div class="alert alert-block alert-warning">
    <h3>Warning</h3>
    The `describe` method will print all the <em>Items</em>, which is greater than 90. This command is somewhat slow and could take up to 60 seconds to complete.
</div>

In [3]:
# List all the Items and their Item IDs
collection.describe()

* <Collection id=noaa-cdr-sea-ice-concentration>
  * <Item id=seaice_conc_daily_nh_2024_v04r00>
  * <Item id=seaice_conc_daily_sh_2024_v04r00>
  * <Item id=seaice_conc_monthly_nh_197811_202406_v04r00>
  * <Item id=seaice_conc_monthly_sh_197811_202406_v04r00>
  * <Item id=seaice_conc_daily_sh_2023_v04r00>
  * <Item id=seaice_conc_daily_sh_2022_v04r00>
  * <Item id=seaice_conc_daily_sh_2021_v04r00>
  * <Item id=seaice_conc_daily_sh_2020_v04r00>
  * <Item id=seaice_conc_daily_sh_2019_v04r00>
  * <Item id=seaice_conc_daily_sh_2018_v04r00>
  * <Item id=seaice_conc_daily_sh_2017_v04r00>
  * <Item id=seaice_conc_daily_sh_2016_v04r00>
  * <Item id=seaice_conc_daily_sh_2015_v04r00>
  * <Item id=seaice_conc_daily_sh_2014_v04r00>
  * <Item id=seaice_conc_daily_sh_2013_v04r00>
  * <Item id=seaice_conc_daily_sh_2012_v04r00>
  * <Item id=seaice_conc_daily_sh_2011_v04r00>
  * <Item id=seaice_conc_daily_sh_2010_v04r00>
  * <Item id=seaice_conc_daily_sh_2009_v04r00>
  * <Item id=seaice_conc_daily_sh_20

#### Print Out *Collection* Metadata

In the next few code cells, you will see different ways to extract metadata information about the STAC *Collection*. You can view all the metadata by simply calling the instance of the collection.

<div class="alert alert-block alert-success">
<h3><strong>Explore</strong></h3>
After you run the command in the next cell, you will see output about the <em>Collection</em>. Lines with a triangle next to them are clickable and reveal more metadata elements underneath. Click these and explore the contents of the <em>Collection</em>.
</div>

In [4]:
collection

<Collection id=noaa-cdr-sea-ice-concentration>

#### Print Specific *Collection* Attributes

In the cell above, you saw how to view all the metadata attributes of the *Collection* at once. You can also view them individually. In the cell below, print out the `id`, `title`, and `description` attributes.

In [5]:
# Print the title and description of the collection
print('Collection ID : ',collection.id)
print('Title: ',collection.title)
print('Descriptbion: ',collection.description)

Collection ID :  noaa-cdr-sea-ice-concentration
Title:  Sea Ice Concentration CDR
Descriptbion:  The Sea Ice Concentration Climate Data Record (CDR) provides a consistent daily and monthly time series of sea ice concentrations for both the north and south Polar Regions on a 25 km x 25 km grid. These data can be used to estimate how much of the ocean surface is covered by ice, and monitor changes in sea ice concentration. The CDR combines concentration estimates using two algorithms developed at the NASA Goddard Space Flight Center (GSFC). Gridded brightness temperatures acquired from a number of Defense Meteorological Satellite Program (DMSP) passive microwave radiometers provide the necessary input to produce the dataset.


#### Print STAC Version
Print out the version of STAC that this *Collection* is using. To do this, you need to use the `get_stac_version` method.

In [6]:
# Print STAC Version
print('STAC Versions: ', pystac.get_stac_version())

STAC Versions:  1.0.0


### Extract All *Items*
Extract all the *Items* from the `collection` variable into a variable called `items` and print out how many there are.

In [7]:
# Extract items from the collection and print out how many there are
items = list(collection.get_all_items())
print(f"Number of items: {len(items)}")

Number of items: 94


### Explore an *Item*

Let's explore one *Item* using the `Item ID` and examine its metadata. In this example, you will explore the 2023 daily Southern Hemisphere *Item*. First, you will need to extract the *Item* from the *Collection*.

#### Extract 2023 Daily Southern Hemisphere *Item*
To do this, you will do the following steps:
* Extract all the `item ids` from the `item` list created in the previous code cell.
* Find the `item id` for the daily 2023 Southern Hemisphere (SH) *Item*. Since you know the form of the `item ids` from the `describe` method used earlier, you will search the `item_ids` for `"daily_sh_2023"`
* Use the `get_item` method and the `item id` to extract it.

In [8]:
# Extract all the Item IDs from the item list
item_ids = [i.id for i in items]

# Find the item id for the 2023 daily SH file
# This command returns a list with one element that contains the id you want
daily_sh_2023_id = [i for i in item_ids if "daily_sh_2023" in i]

# Get item using item id
daily_sh_2023_item = collection.get_item(daily_sh_2023_id[0], recursive=True)

#### Print *Item* Metadata
The `daily_sh_2023_item` contains metadata in the form of properties, attributes, and assets. You will explore some of these by printing them.

First, look at all metadata for the `daily_sh_2023_item`. 

<div class="alert alert-block alert-success">
<h3><strong>Explore</strong></h3>
Similarly to when you printed out the <em>Collection</em> attributes, you will see similar output when you print out the <em>Item</em>. The lines with a triangle next to them are clickable and reveal more metadata elements underneath. Click these and explore the contents of the <em>Item</em>.
</div>

In [9]:
# Show all properites, attributes, and assets by looking at the item instance
daily_sh_2023_item

<Item id=seaice_conc_daily_sh_2023_v04r00>

#### Print Out the *Item* Properties
You can also look at all the *Item* properties with the following command.

In [10]:
daily_sh_2023_item.properties

{'start_datetime': '2023-01-01T00:00:00Z',
 'end_datetime': '2023-12-31T23:59:59Z',
 'noaa_cdr:interval': 'daily',
 'processing:level': 'L3',
 'proj:epsg': None,
 'proj:wkt2': 'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",6378273,298.279411123064,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Polar Stereographic (variant B)",ID["EPSG",9829]],PARAMETER["Latitude of standard parallel",-70,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8832]],PARAMETER["Longitude of origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8833]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",north,MERIDIAN[90,ANGLEUNIT["degree",0.0174532925199433,ID["EPSG",9122]]],ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,MERIDIAN[0,ANGLEUNIT["degree",0

#### Print Out Other *Item* Properties and Attributes
You can print specific properties and attributes with the next commands.

In [11]:
print('Geometry: ', daily_sh_2023_item.geometry)
print('Geospatial Bounding Box: ', daily_sh_2023_item.bbox)
print('Start date: ', daily_sh_2023_item.properties['start_datetime'])
print('End date: ', daily_sh_2023_item.properties['end_datetime'])

Geometry:  {'type': 'Polygon', 'coordinates': [[[180.0, -90.0], [180.0, -41.45], [-180.0, -41.45], [-180.0, -90.0], [180.0, -90.0]]]}
Geospatial Bounding Box:  [-180.0, -90.0, 180.0, -41.45]
Start date:  2023-01-01T00:00:00Z
End date:  2023-12-31T23:59:59Z


<div class="alert alert-block alert-success">
<h3><strong>Exercise 1</strong></h3>
Using the example from Tutorial 1, uncomment the template below, and change the code as necessary to extract and explore the metadata for the 2012 daily <em>Item</em> for the Northern Hemisphere. </br></br>
<strong>Be sure to update variable names as appropriate. For example, if there is a year in the variable name or the hemisphere designation (<em>sh</em> or <em>nh</em>), update that accordingly.</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Hint:</strong> Think about what you need to search for to find the correct `Item id` for the daily 2012 Northern Hemisphere <em>Item</em>.
</div>


In [12]:
# Find the item id for the 2012 daily NH file
#daily_sh_2023_id = [i for i in item_ids if "daily_sh_2023" in i] # Uncomment out this command and update as necessary

# Get item using item id
#daily_sh_2023_item = collection.get_item(daily_sh_2023_id[0], recursive=True) # Uncomment out this command and update as necessary

# Show all properites, attributes, and assets by looking at the item instance
# Note that these metadata elements are the same as those for the 2023 SH item but with different values
#daily_sh_2023_item # Uncomment out this command and update as necessary

<Item id=seaice_conc_daily_nh_2012_v04r00>

In [13]:
# Uncomment out this command and update as necessary
#print('Geometry: ', daily_sh_2023_item.geometry) # Uncomment out this command and update as necessary
#print('Geospatial Bounding Box: ', daily_sh_2023_item.bbox) # Uncomment out this command and update as necessary
#print('Start date: ', daily_sh_2023_item.properties['start_datetime']) # Uncomment out this command and update as necessary
#print('End date: ', daily_sh_2023_item.properties['end_datetime']) # Uncomment out this command and update as necessary

Geometry:  {'type': 'Polygon', 'coordinates': [[[180.0, 31.35], [180.0, 90.0], [-180.0, 90.0], [-180.0, 31.35], [180.0, 31.35]]]}
Geospatial Bounding Box:  [-180.0, 31.35, 180.0, 90.0]
Start date:  2012-01-01T00:00:00Z
End date:  2012-12-31T23:59:59Z


<div class="alert alert-block alert-success">
<h3><strong>Bonus Question</strong></h3>
What command would you use to view the netCDF assets? Add this command to the blank line in the code cell below. You'll be using this in Part 2 of the tutorial below, so if you don't know the answer, you can find it there.
</div>

<div class="alert alert-block alert-info">
<strong>Hint:</strong> If you are unsure about what attributes are available, print out the <em>Item</em> instance again. The command will look something like <code>item_name.attribute['element']</code>.
</div>

In [14]:
# Add a command to view the netCDF assets here


************************
## Tutorial Part 2

Read NetCDF File from the STAC
************************

### Read Data from NetCDF File Using STAC
Now that you have explored the SIC CDR STAC *Collection* and its *Items*, you can use this information to read the netCDF data file associated with an *Item*. You will read the data from the 2023 daily Southern Hemisphere *Item* extracted in Part 1 of this tutorial.

#### Print out the NetCDF Assets
The STAC *Item* contains metadata attributes called assets that give you some information about the data in the netCDF file including the location of the file via its URL in the `href` attribute.

<div class="alert alert-block alert-info">
<strong>Note:</strong> The command in the next code cell is the answer to the Bonus Question from Exercise 1.
</div>

In [15]:
# Access the NetCDF assets from the STAC *Item* and see the components
daily_sh_2023_item.assets['netcdf']

<Asset href=https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc>

#### Extract the URL from the Assets and Read the NetCDF File

You will use fsspec and xarray to read the netCDF file. To get the URL of the netCDF file, you will extract the `href` attribute from the netCDF assets. 

In [16]:
# Get the URL from the STAC Item assets and print it out
netcdf_url_2023 = daily_sh_2023_item.assets['netcdf'].href
print("NetCDF URL: ", netcdf_url_2023)

# Read the netCDF file at that URL using fsspec and xarray
fs_2023 = fsspec.filesystem('https')
ds_2023 = xr.open_dataset(fs_2023.open(netcdf_url_2023))

NetCDF URL:  https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc


#### Explore the NetCDF Dataset and its Variables
You can view all the variables within the netCDF file by calling the dataset instance. Xarray provides a nice interactive way to explore the file contents. You can see the coordinates for the data (`time`, `xgrid`, `ygrid`) and then the variables themselves (`cdr_sesaice_conc`, `projection`, `qa_of_cdr_seaice_conc`, `stdev_of_cdr_seaice_conc`, etc.).

<div class="alert alert-block alert-success">
<h3><strong>Explore</strong></h3>
After running the command in the code cell below, you will see information about the netCDF data formatted with Xarray. The first line, `Dimensions`, provides the dimensions of the dataset. In this case, `tdim` is the time dimension with 365 values (one for each days of the year), the y axis with 332 elements, and the x axis with 316 elements.</br></br>
This output also provides information about the netCDF variables. Each netCDF variable has two icons to its right: a file icon and a database icon (looks like 3 coins stacked on top of each other).
    <ul>
    <li>File icon: Clicking this lists the variable attributes available for that variable (for example: long_name, standard_name, flag_values, units, valid range, etc.). These variable attributes can help you determine what a variable is and how to use it. Note that not all variables have the same attributes. These are dependent on what the data creator supplied with the netCDF data files.</li>
    <li>Database icon: Clicking this shows you the number of elements in the variable and its data type.</li>
    </ul>
Click on the icons to explore the variable attributes of the netCDF variables. <strong>Note:</strong> To close the content, you have to click the icon again.
</br></br>
In addition to variable attributes, the netCDF files also contain global attributes. These can be viewed by clicking on the word `Attributes` in the xarray output. You will see things like title, summary, and spatial and temporal resolution, to name a few. 
</div>

In [17]:
# Print out the dataset instance to see its contents
ds_2023

<xarray.Dataset> Size: 1GB
Dimensions:                      (tdim: 365, y: 332, x: 316)
Coordinates:
    time                         (tdim) datetime64[ns] 3kB ...
    xgrid                        (x) float32 1kB ...
    ygrid                        (y) float32 1kB ...
Dimensions without coordinates: tdim, y, x
Data variables:
    cdr_seaice_conc              (tdim, y, x) float32 153MB ...
    nsidc_bt_seaice_conc         (tdim, y, x) float32 153MB ...
    nsidc_nt_seaice_conc         (tdim, y, x) float32 153MB ...
    projection                   |S1 1B ...
    qa_of_cdr_seaice_conc        (tdim, y, x) float32 153MB ...
    spatial_interpolation_flag   (tdim, y, x) float32 153MB ...
    stdev_of_cdr_seaice_conc     (tdim, y, x) float32 153MB ...
    temporal_interpolation_flag  (tdim, y, x) float32 153MB ...
    latitude                     (y, x) float32 420kB ...
    longitude                    (y, x) float32 420kB ...
Attributes: (12/39)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     NOAA/NSIDC Climate Data Record of Passive Micr...
    references:                Comiso, J. C., and F. Nishio. 2008. Trends in ...
    comment:                   none
    program:                   NOAA Climate Data Record Program
    cdr_variable:              cdr_seaice_conc
    ...                        ...
    acknowledgment:            This project was supported in part by a grant ...
    platform:                  DMSP 5D-3/F17 > Defense Meteorological Satelli...
    sensor:                    SSMI/S > Special Sensor Microwave Imager/Sounder
    date_created:              2023-09-26T20:17:19Z
    time_coverage_start:       2023-01-01T00:00:00Z
    time_coverage_end:         2023-12-31T23:59:59Z

#### Access Specific NetCDF Global Attributes
You can print out specific global attributes by calling them by name from the dataset instance. As noted above, you can find the name of these attributes by clicking `Attributes` in the code output above. In the code cell below, you will print out the `spatial_resolution`, `platform`, and `sensor` from the global attributes.

In [18]:
# Print some global attributes
print("Spatial Resolution: ", ds_2023.spatial_resolution)
print("Platform: ", ds_2023.platform)
print("Sensor: ", ds_2023.sensor)

Spatial Resolution:  25km
Platform:  DMSP 5D-3/F17 > Defense Meteorological Satellite Program-F17
Sensor:  SSMI/S > Special Sensor Microwave Imager/Sounder


#### Other Ways to Explore the NetCDF Variables
To print out a simple list of the netCDF variables, you can use the `data_vars` property of the xarray dataset. One of the variables is called `cdr_seaice_conc`. This contains the sea ice data you are interested in.

In [19]:
# Print out simple list of netCDF variables
ds_2023.data_vars

Data variables:
    cdr_seaice_conc              (tdim, y, x) float32 153MB ...
    nsidc_bt_seaice_conc         (tdim, y, x) float32 153MB ...
    nsidc_nt_seaice_conc         (tdim, y, x) float32 153MB ...
    projection                   |S1 1B ...
    qa_of_cdr_seaice_conc        (tdim, y, x) float32 153MB ...
    spatial_interpolation_flag   (tdim, y, x) float32 153MB ...
    stdev_of_cdr_seaice_conc     (tdim, y, x) float32 153MB ...
    temporal_interpolation_flag  (tdim, y, x) float32 153MB ...
    latitude                     (y, x) float32 420kB ...
    longitude                    (y, x) float32 420kB ...

#### Print Out All Metadata for the Variables
You can view all the information at once about the netCDF variables using the `keys` method on the `variables` property.
<div class="alert alert-block alert-warning">
    <h3>Warning</h3>
    This prints a rather messy list of all the variables and their attributes.
</div>

In [20]:
# Take a look at the variable keys for the netCDF dataset
ds_2023.variables.keys()

KeysView(Frozen({'cdr_seaice_conc': <xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc
    valid_range:          [  0 100]
    cell_methods:         tdim: mean, 'nsidc_bt_seaice_conc': <xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:      Passive Microwave Daily Southern Hemisphere Sea Ice Conce...
    standard_name:  sea_ice_area_fraction
    units:   

#### Extract the Sea Ice Concentration Data

You saw from the command above that the sea ice concentration variable is called `cdr_seaice_conc`. 
You will use this to extract the data from the file into a variable.

In [21]:
# Pull out sea ice concentration variable as an xarray object
cdr_seaice_conc_2023 = ds_2023.variables['cdr_seaice_conc']

#### Explore the Attributes of the cdr_seaice_conc Variable
Just as you were able to view the attribute information for the variables in the xarray dataset object `ds_2023`, you can also view the attributes specifically for the variable once you've extracted it. As with the xarray dataset, viewing the properties of the xarray variable can be done in several ways. You will look at two of them: calling the xarray variable instance and using the `attrs` property of the xarray varaible.

<div class="alert alert-block alert-info">
<strong>Hint:</strong> You'll need some of this information in Notebook 3 when you plot the data.
</div>

**Using the Xarray Variable Instance**

You can simply call the name of the xarray variable to view its properties.

<div class="alert alert-block alert-success">
<h3><strong>Explore</strong></h3>
Click on the word <code>Attributes</code> and explore the attributes associated with the variable.
</div>

In [22]:
# Print out the cdr_seaice_conc_2023 Xarray variable instance
cdr_seaice_conc_2023

<xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    standard_name:        sea_ice_area_fraction
    units:                1
    flag_values:          [251 252 253 254 255]
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:         projection
    reference:            https://nsidc.org/data/g02202/versions/4/
    ancillary_variables:  stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc
    valid_range:          [  0 100]
    cell_methods:         tdim: mean

**Using the attrs Property**

You can simply print the attributes as text using the `attrs` property.

In [23]:
cdr_seaice_conc_2023.attrs

{'long_name': 'NOAA/NSIDC Climate Data Record of Passive Microwave Daily Southern Hemisphere Sea Ice Concentration',
 'standard_name': 'sea_ice_area_fraction',
 'units': '1',
 'flag_values': array([251, 252, 253, 254, 255], dtype=uint8),
 'flag_meanings': 'pole_hole lakes coastal land_mask missing_data',
 'datum': '+ellps=urn:ogc:def:crs:EPSG::4326',
 'grid_mapping': 'projection',
 'reference': 'https://nsidc.org/data/g02202/versions/4/',
 'ancillary_variables': 'stdev_of_cdr_seaice_conc qa_of_cdr_seaice_conc',
 'valid_range': array([  0, 100], dtype=uint8),
 'cell_methods': 'tdim: mean'}

<div class="alert alert-block alert-success">
<h3><strong>Exercise 2</strong></h3>
Using the example from Tutorial 2, uncomment the template below, and change the code as necessary to read the 2012 daily file for the Northern Hemisphere and extract the temporal interpolation flag variable and explore its contents.</br></br>
<strong>Be sure to update variable names as appropriate.</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Hint:</strong> Use the <em>Item</em> variable you created in Exercise 1 to accomplish this task. Use the method object <code>data_vars</code> to find the name of the flag.
</div>

#### Read the 2012 Norhtern Hemisphere NetCDF File

In [24]:
# Exercise
# Get the URL from the STAC Item assets and print it out
#netcdf_url_2023 = daily_sh_2023_item.assets['netcdf'].href # Uncomment out this command and update as necessary
#print("NetCDF URL: ", netcdf_url_2023) # Uncomment out this command and update as necessary

# Read the netCDF file at that URL using fsspec and xarray and print out the result
#fs_2023 = fsspec.filesystem('https') # Uncomment out this command and update as necessary
#ds_2023 = xr.open_dataset(fs_2023.open(netcdf_url_2023)) # Uncomment out this command and update as necessary

NetCDF URL:  https://noaadata.apps.nsidc.org/NOAA/G02202_V4/north/aggregate/seaice_conc_daily_nh_2012_v04r00.nc


In [ ]:
# Print out simple list of netCDF variables and find the one for the temporal interpolation flag
#ds_2023.data_vars # Uncomment out this command and update as necessary

#### Extract Temporal Interpolation Flag

<div class="alert alert-block alert-info">
<strong>Hint:</strong> From the listing of the variables in the previous code cell, you see that the netCDF variable for the temporal interpolatoin flag is called <code>temporal_interpolation_flag</code>.
</div>

In [25]:
# Pull out temporal interploation flag into a variable as an xarray object
# Note: The variable name has been set for you but you must update all the information on the right side of the equals sign
# with the correct xarray dataset name and netcdf variable name.
#temporal_flag = ds_2023.variables['netcdf_variable'] # Uncomment out this command and update as necessary

# Print out the attribute details of the temporal interpolation flag
#temporal_flag # Uncomment out this command

<xarray.Variable (tdim: 365, y: 332, x: 316)> Size: 153MB
[38292880 values with dtype=float32]
Attributes:
    long_name:      Passive Microwave Daily Southern Hemisphere Sea Ice Conce...
    standard_name:  interpolation_flag
    flag_masks:     [ 1  2  3  4  5 10 11 20 22 30 33 40 44 50 55]
    flag_meanings:  1_day_following 2_days_following 3_days_following 4_days_...
    comment:        If flagged, the value has been replaced by the value n-da...
    datum:          +ellps=urn:ogc:def:crs:EPSG::4326
    grid_mapping:   projection
    valid_range:    [ 0 55]
    cell_methods:   tdim: mean

### Next Steps

After successful completion of Notebook 2, you can go on to Notebook 3: Mapping Sea Ice Concentration.

### Data Statement

Meier, W. N., Fetterer, F., Windnagel, A. K., & Stewart, J. S. (2021). NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration. (G02202, Version 4). [Data Set]. Boulder, Colorado USA. National Snow and Ice Data Center. https://doi.org/10.7265/efmz-2t65.

### Metadata

* Language/Package(s)
  * Python
  * xarray
  * pystac
* Geophysical Keywords
  * Sea Ice Concentration

### Software License

Copyright 2025 NOAA NCAI

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### Content License

[CC BY 4.0](https://creativecommons.org/licenses/by/4.0/ "Createive Commons BY 4.0 License")

### Disclaimer

This Jupyter notebook is a scientific product and is not official communication of the National Oceanic and Atmospheric Administration, the United States Department of Commerce, the National Snow and Ice Data Center, or the University of Colorado at Boulder. All NOAA Jupyter notebooks are provided on an 'as is' basis, and the user assumes responsibility for its use. Any claims against the Department of Commerce or Department of Commerce bureaus stemming from the use of this Jupyter notebook will be governed by all applicable Federal law. Any reference to specific commercial products, processes, or services by service mark, trademark, manufacturer, or otherwise does not constitute or imply their endorsement, recommendation or favoring by the Department of Commerce. The Department of Commerce seal and logo, or the seal and logo of a DOC bureau, shall not be used in any manner to imply endorsement of any commercial product or activity by DOC or the United States Government.